In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

'''
The my_dir/intro_to_kt directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional overwrite=True argument while instantiating the tuner.
'''


In [2]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()


4422102/4422102 [==============================] - 0s 0us/step


In [3]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [4]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model

In [5]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')



In [6]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


In [7]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 11s]
val_accuracy: 0.8919166922569275

Best val_accuracy So Far: 0.8919166922569275
Total elapsed time: 00h 15m 36s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 416 and the optimal learning rate for the optimizer
is 0.001.


In [8]:
# Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.4945 - accuracy: 0.8250 - val_loss: 0.4055 - val_accuracy: 0.8548
Epoch 2/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3684 - accuracy: 0.8658 - val_loss: 0.3814 - val_accuracy: 0.8625
Epoch 3/50
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3301 - accuracy: 0.8779 - val_loss: 0.3550 - val_accuracy: 0.8731
Epoch 4/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3069 - accuracy: 0.8863 - val_loss: 0.3552 - val_accuracy: 0.8723
Epoch 5/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.2864 - accuracy: 0.8938 - val_loss: 0.3410 - val_accuracy: 0.8790
Epoch 6/50
1500/1500 [==============================] - 7s 4ms/step - loss: 0.2697 - accuracy: 0.8996 - val_loss: 0.3323 - val_accuracy: 0.8810
Epoch 7/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.2561 - accuracy: 0.9035 - val_loss: 0.3168 - val_accuracy:

In [9]:
# Re-instantiate the hypermodel and train it with the optimal number of epochs from above.

hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/29
1500/1500 [==============================] - 8s 5ms/step - loss: 0.4916 - accuracy: 0.8264 - val_loss: 0.3957 - val_accuracy: 0.8562
Epoch 2/29
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3703 - accuracy: 0.8651 - val_loss: 0.3669 - val_accuracy: 0.8630
Epoch 3/29
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3316 - accuracy: 0.8783 - val_loss: 0.3412 - val_accuracy: 0.8766
Epoch 4/29
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3073 - accuracy: 0.8865 - val_loss: 0.3592 - val_accuracy: 0.8733
Epoch 5/29
1500/1500 [==============================] - 7s 5ms/step - loss: 0.2862 - accuracy: 0.8940 - val_loss: 0.3293 - val_accuracy: 0.8828
Epoch 6/29
1500/1500 [==============================] - 7s 4ms/step - loss: 0.2710 - accuracy: 0.8989 - val_loss: 0.3175 - val_accuracy: 0.8853
Epoch 7/29
1500/1500 [==============================] - 7s 4ms/step - loss: 0.2552 - accuracy: 0.9047 - val_loss: 0.3284 - val_accuracy:

In [10]:
# To finish this tutorial, evaluate the hypermodel on the test data.

eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 2s 5ms/step - loss: 0.4609 - accuracy: 0.8853
[test loss, test accuracy]: [0.46086248755455017, 0.8852999806404114]


In [11]:
class SharpeValidationLoss(keras.callbacks.Callback):
    # TODO check if weights already exist and pass in best sharpe
    def __init__(
            self,
            inputs,
            returns,
            time_indices,
            num_time,  # including a count for nulls which will be indexed as 0
            early_stopping_patience,
            n_multiprocessing_workers,
            weights_save_location="tmp/checkpoint",
            # verbose=0,
            min_delta=1e-4,
    ):
        super(keras.callbacks.Callback, self).__init__()
        self.inputs = inputs
        self.returns = returns
        self.time_indices = time_indices
        self.n_multiprocessing_workers = n_multiprocessing_workers
        self.early_stopping_patience = early_stopping_patience
        self.num_time = num_time
        self.min_delta = min_delta

        self.best_sharpe = np.NINF  # since calculating positive Sharpe...
        # self.best_weights = None
        self.weights_save_location = weights_save_location
        # self.verbose = verbose

    def set_weights_save_loc(self, weights_save_location):
        self.weights_save_location = weights_save_location

    def on_train_begin(self, logs=None):
        self.patience_counter = 0
        self.stopped_epoch = 0
        self.best_sharpe = np.NINF

    def on_epoch_end(self, epoch, logs=None):
        positions = self.model.predict(
            self.inputs,
            workers=self.n_multiprocessing_workers,
            use_multiprocessing=True,  # , batch_size=1
        )

        captured_returns = tf.math.unsorted_segment_mean(
            positions * self.returns, self.time_indices, self.num_time
        )[1:]
        # ignoring null times

        # TODO sharpe
        sharpe = (
                tf.reduce_mean(captured_returns)
                / tf.sqrt(
            tf.math.reduce_variance(captured_returns)
            + tf.constant(1e-9, dtype=tf.float64)
        )
                * tf.sqrt(tf.constant(252.0, dtype=tf.float64))
        ).numpy()
        if sharpe > self.best_sharpe + self.min_delta:
            self.best_sharpe = sharpe
            self.patience_counter = 0  # reset the count
            # self.best_weights = self.model.get_weights()
            self.model.save_weights(self.weights_save_location)
        else:
            # if self.verbose: #TODO
            self.patience_counter += 1
            if self.patience_counter >= self.early_stopping_patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                self.model.load_weights(self.weights_save_location)
        logs["sharpe"] = sharpe  # for keras tuner
        print(f"\nval_sharpe {logs['sharpe']}")


In [12]:
#data, labels, active_flags, _, _ = ModelFeatures._unpack(train_data)
#val_data, val_labels, val_flags, _, val_time = ModelFeatures._unpack(valid_data)

callbacks = [
    SharpeValidationLoss(
        val_data,
        val_labels,
        val_time_indices,
        num_val_time,
        self.early_stopping_patience,
        self.n_multiprocessing_workers,
    ),
    tf.keras.callbacks.TerminateOnNaN(),
]


NameError: name 'val_data' is not defined